In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
%matplotlib inline

# 관련 라이브러리 임포트 
import matplotlib.font_manager as fm

#  한글글꼴로 변경
# plt.rcParams['font.family'] = '한글글꼴명'
plt.rcParams['font.size'] = 11.0
# plt.rcParams['font.family'] = 'batang'
plt.rcParams['font.family'] = 'Malgun Gothic'

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
matplotlib.rcParams['axes.unicode_minus'] = False

# 그래프 기본 크기 설정 
plt.rcParams['figure.figsize'] = [10, 6]

import warnings
warnings.filterwarnings('ignore')

In [2]:
base = pd.read_excel('data/ID있는버전.xlsx')
base.head(5)

,ID,시군구,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,거래유형
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,80,201801,3,130000,4,1987,언주로 3,-
1,서울특별시 동작구 신대방동 롯데관악타워 150,서울특별시 동작구 신대방동,롯데관악타워,150,201801,4,74000,27,1996,보라매로5길 51,-
2,서울특별시 동작구 신대방동 경남교수 80,서울특별시 동작구 신대방동,경남교수,80,201801,25,65000,17,2001,여의대방로10길 14,-
3,서울특별시 동작구 상도동 힐스테이트 상도 프레스티지 80,서울특별시 동작구 상도동,힐스테이트 상도 프레스티지,80,201801,27,85800,15,2013,상도로 346-2,-
4,서울특별시 동작구 상도동 힐스테이트 상도 프레스티지 120,서울특별시 동작구 상도동,힐스테이트 상도 프레스티지,120,201801,24,104500,12,2013,상도로 346-2,-


In [3]:
base['면적당매매가'] = base['거래금액(만원)'] / base['전용면적(㎡)']

In [4]:
base['저층여부'] = np.where(base['층']<5, '저층', '고층')
base.head(1)

,ID,시군구,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,거래유형,면적당매매가,저층여부
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,80,201801,3,130000,4,1987,언주로 3,-,1625.0,저층


In [5]:
del base['계약일']
base.head(1)

,ID,시군구,단지명,전용면적(㎡),계약년월,거래금액(만원),층,건축년도,도로명,거래유형,면적당매매가,저층여부
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,80,201801,130000,4,1987,언주로 3,-,1625.0,저층


In [6]:
base.drop(base[base['거래유형']=='직거래'].index)
base.head(1)

,ID,시군구,단지명,전용면적(㎡),계약년월,거래금액(만원),층,건축년도,도로명,거래유형,면적당매매가,저층여부
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,80,201801,130000,4,1987,언주로 3,-,1625.0,저층


In [7]:
base2 = pd.read_excel('data/bigreant.xlsx')
base2.head(1)

,Unnamed: 0,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명,계약기간,계약구분,갱신요구권 사용,종전계약 보증금 (만원),종전계약 월세 (만원)
0,0,서울특별시 강남구 개포동,12,12.0,0.0,SH대치1단지아파트,월세,25.35,201803,29,"3,865",17,11,1991.0,개포로109길 5,-,-,-,,


In [8]:
base2['계약년월'] = pd.to_datetime(base2['계약년월'], format='%Y%m')
base2['분기'], base2['년도'] = (base2['계약년월'].dt.quarter, base2['계약년월'].dt.year)
base2['분기'] = base2[['년도','분기']].apply(lambda row:'/'.join(row.values.astype(str)), axis=1)
del base2['년도']
del base2['Unnamed: 0']
base2.head(1)

,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명,계약기간,계약구분,갱신요구권 사용,종전계약 보증금 (만원),종전계약 월세 (만원),분기
0,서울특별시 강남구 개포동,12,12.0,0.0,SH대치1단지아파트,월세,25.35,2018-03-01,29,"3,865",17,11,1991.0,개포로109길 5,-,-,-,,,2018/1


In [9]:
base2.drop(['계약기간','계약구분','갱신요구권 사용','종전계약 보증금 (만원)','종전계약 월세 (만원)'], axis=1, inplace=True)
base2.head(1)

,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명,분기
0,서울특별시 강남구 개포동,12,12.0,0.0,SH대치1단지아파트,월세,25.35,2018-03-01,29,"3,865",17,11,1991.0,개포로109길 5,2018/1


In [10]:
base2['단지명'] = base2['단지명'].str.replace(r'\([^)]*\)', '', regex=True)
base2.head(1)

,시군구,번지,본번,부번,단지명,전월세구분,전용면적(㎡),계약년월,계약일,보증금(만원),월세(만원),층,건축년도,도로명,분기
0,서울특별시 강남구 개포동,12,12.0,0.0,SH대치1단지아파트,월세,25.35,2018-03-01,29,"3,865",17,11,1991.0,개포로109길 5,2018/1


In [11]:
base2['전용면적(㎡)'] = base2['전용면적(㎡)'].round(-1)
base2.drop(['번지','본번','부번','계약일'], axis=1, inplace=True)
base2.to_excel('data/NewRent.xlsx')

In [16]:
base2 = pd.read_excel('data/NewRent.xlsx')
base2['ID'] = base2['시군구'] + ' ' + base2['단지명'] + ' ' + base2['전용면적(㎡)'].astype('str')
base2.head(5)

,Unnamed: 0,시군구,단지명,전월세구분,전용면적(㎡),계약년월,보증금(만원),월세(만원),층,건축년도,도로명,분기,ID
0,0,서울특별시 강남구 개포동,SH대치1단지아파트,월세,30,2018-03-01,"3,865",17,11,1991.0,개포로109길 5,2018/1,서울특별시 강남구 개포동 SH대치1단지아파트 30
1,1,서울특별시 강남구 개포동,개포2차현대아파트,월세,80,2018-02-01,"39,000",30,4,1988.0,언주로 103,2018/1,서울특별시 강남구 개포동 개포2차현대아파트 80
2,2,서울특별시 강남구 개포동,개포2차현대아파트,전세,80,2018-04-01,"52,000",0,9,1988.0,언주로 103,2018/2,서울특별시 강남구 개포동 개포2차현대아파트 80
3,3,서울특별시 강남구 개포동,개포2차현대아파트,전세,80,2018-05-01,"50,000",0,5,1988.0,언주로 103,2018/2,서울특별시 강남구 개포동 개포2차현대아파트 80
4,4,서울특별시 강남구 개포동,개포2차현대아파트,전세,80,2018-05-01,"56,000",0,9,1988.0,언주로 103,2018/2,서울특별시 강남구 개포동 개포2차현대아파트 80


In [17]:
base2.drop(['Unnamed: 0','시군구','단지명','전용면적(㎡)','계약년월','건축년도','도로명'], axis=1, inplace=True)

In [23]:
del base2['층']
base2.head(5)

,전월세구분,보증금(만원),월세(만원),분기,ID
0,월세,"3,865",17,2018/1,서울특별시 강남구 개포동 SH대치1단지아파트 30
1,월세,"39,000",30,2018/1,서울특별시 강남구 개포동 개포2차현대아파트 80
2,전세,"52,000",0,2018/2,서울특별시 강남구 개포동 개포2차현대아파트 80
3,전세,"50,000",0,2018/2,서울특별시 강남구 개포동 개포2차현대아파트 80
4,전세,"56,000",0,2018/2,서울특별시 강남구 개포동 개포2차현대아파트 80


In [22]:
base['계약년월'] = pd.to_datetime(base['계약년월'], format='%Y%m')
base['분기'], base['년도'] = (base['계약년월'].dt.quarter, base['계약년월'].dt.year)
base['분기'] = base[['년도','분기']].apply(lambda row:'/'.join(row.values.astype(str)), axis=1)
del base['년도']
base.head(5)

,ID,시군구,단지명,전용면적(㎡),계약년월,거래금액(만원),층,건축년도,도로명,거래유형,면적당매매가,저층여부,분기
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,80,2018-01-01,130000,4,1987,언주로 3,-,1625.000000,저층,2018/1
1,서울특별시 동작구 신대방동 롯데관악타워 150,서울특별시 동작구 신대방동,롯데관악타워,150,2018-01-01,74000,27,1996,보라매로5길 51,-,493.333333,고층,2018/1
2,서울특별시 동작구 신대방동 경남교수 80,서울특별시 동작구 신대방동,경남교수,80,2018-01-01,65000,17,2001,여의대방로10길 14,-,812.500000,고층,2018/1
3,서울특별시 동작구 상도동 힐스테이트 상도 프레스티지 80,서울특별시 동작구 상도동,힐스테이트 상도 프레스티지,80,2018-01-01,85800,15,2013,상도로 346-2,-,1072.500000,고층,2018/1
4,서울특별시 동작구 상도동 힐스테이트 상도 프레스티지 120,서울특별시 동작구 상도동,힐스테이트 상도 프레스티지,120,2018-01-01,104500,12,2013,상도로 346-2,-,870.833333,고층,2018/1


In [26]:
# 분명히 잘못됨
bigbase = pd.merge(base, base2, left_on='ID', right_on='ID', how='left')
bigbase

,ID,시군구,단지명,전용면적(㎡),계약년월,거래금액(만원),층,건축년도,도로명,거래유형,면적당매매가,저층여부,분기_x,전월세구분,보증금(만원),월세(만원),분기_y
0,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,80,2018-01-01,130000,4,1987,언주로 3,-,1625.0,저층,2018/1,전세,"43,000",0,2018/1
1,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,80,2018-01-01,130000,4,1987,언주로 3,-,1625.0,저층,2018/1,전세,"39,000",0,2018/1
2,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,80,2018-01-01,130000,4,1987,언주로 3,-,1625.0,저층,2018/1,전세,"45,000",0,2018/1
3,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,80,2018-01-01,130000,4,1987,언주로 3,-,1625.0,저층,2018/1,전세,"41,500",0,2018/1
4,서울특별시 강남구 개포동 개포6차우성아파트1동~8동 80,서울특별시 강남구 개포동,개포6차우성아파트1동~8동,80,2018-01-01,130000,4,1987,언주로 3,-,1625.0,저층,2018/1,전세,"45,000",0,2018/1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63206849,서울특별시 용산구 문배동 용산큐브 10,서울특별시 용산구 문배동,용산큐브,10,2022-06-01,20000,17,2013,백범로90길 12,중개거래,2000.0,고층,2022/2,월세,"1,000",70,2022/2
63206850,서울특별시 용산구 문배동 용산큐브 10,서울특별시 용산구 문배동,용산큐브,10,2022-06-01,20000,17,2013,백범로90길 12,중개거래,2000.0,고층,2022/2,월세,800,68,2022/2
63206851,서울특별시 용산구 문배동 용산큐브 10,서울특별시 용산구 문배동,용산큐브,10,2022-06-01,20000,17,2013,백범로90길 12,중개거래,2000.0,고층,2022/2,월세,500,70,2022/2
63206852,서울특별시 용산구 문배동 용산큐브 10,서울특별시 용산구 문배동,용산큐브,10,2022-06-01,20000,17,2013,백범로90길 12,중개거래,2000.0,고층,2022/2,월세,"1,000",68,2022/2
